In [ ]:
import re
import pandas as pd

In [2]:
class Attribute:
    def __init__(self, attrib):
        self.attrib = attrib

class Object: # Menu와 관한 Database 객체 (속성, default_option)
    def __init__(self, menu_name="TBD", temperature="TBD", size="TBD", shot="TBD", ice_amount="TBD", whipping="TBD", attributes=[]):
        self.name = menu_name
        self.attrib_list = [Attribute(attrib) for attrib in attributes]
        self.default_EO3 = temperature
        self.default_EO4 = size
        self.default_DO1 = shot
        self.default_DO2 = ice_amount
        self.default_CO1 = whipping

class Order: # 주문을 저장하는 객체
    def __init__(self, menu_name, quantity, temperature, size, extra_shot=0):
        self.EO1 = menu_name
        self.EO2 = quantity
        self.EO3 = temperature
        self.EO4 = size
        self.DO1 = extra_shot

In [18]:
class ExpertSystem:
    def __init__(self):
        df = pd.read_excel("./menu_default_info.xlsx") #menu default value informations
        self.menu_base = []
        
        for index, row in df.iterrows(): # Object 인스턴스 생성 후 menu_base 추가
            obj = Object(
                menu_name = row['menu_name'],
                size = row['size'],
                temperature = row['temperature'],
                shot = row['shot'],
                ice_amount = row['ice_amount'],
                whipping = row['whipping'],
                attributes = row['attributes'].split(", ")  # 속성을 문자열에서 리스트로 변환
            )
            self.menu_base.append(obj)
            
        self.yes = []
        self.no = []
        self.orders = []  # 주문 정보를 저장하는 큐
        
    def reset_yes_no(self):
        # 사용자의 선호도에 대한 답변 초기화 함수
        self.yes = []
        self.no = []
        
    def enter(self):
        # 전문가가 특성을 입력하는 과정
        while True:
            name = input("메뉴 이름: ") # 특성을 입력할 메뉴 이름
            if not name:
                break
            print(f'<{name} 특성 입력 모드 >\n* 끝내려면 enter 입력, 마지막 요소 지우기는 "수정"\n')
            attributes = []
            while True:
                attrib = input("특성 입력: ") # 해당 메뉴의 특성 입력
                if not attrib:
                    break
                if attrib == '수정': # 해당 메뉴의 바로 이전에 등록된 특성 제거 기능
                    if len(obj.attrib_list) != 0: # 바로 이전에 입력된 특성 제거
                        obj.attrib_list.pop()
                        continue
                    else:
                        print(f'{obj.name}를 지웁니다.') # 특성이 하나도 입력되지 않으면 해당 메뉴 제거.
                        del obj
                        break
                attributes.append(attrib)
            self.menu_base.append(Object(name, attributes))

    def query(self): #음료를 찾아 반환하는 query 함수
        for obj in self.menu_base:
            if self.try_object(obj):
                print(f"찾는 음료는 {obj.name} 입니다.")
                return
        print("적당한 음료를 찾지 못했습니다.")
        
    # ExpertSystem 인스턴스에서 menu_base에 있는 모든 메뉴 이름을 추출하여 정규 표현식 문자열 생성
    def create_menu_pattern(expert_system):
        menu_names = [obj.name for obj in expert_system.menu_base]
        menu_pattern_str = "|".join(menu_names)
        return re.compile(menu_pattern_str)

    # 한글 숫자를 실제 숫자로 변환하는 함수
    @staticmethod
    def convert_to_number(match):
        word_to_number = {
            "하나": "1", "둘": "2", "셋": "3", "넷": "4",
            "다섯": "5", "여섯": "6", "일곱": "7", "여덟": "8",
            "아홉": "9", "열": "10"
        }
        if match.group() in word_to_number:
            return word_to_number[match.group()]
        else:
            return match.group()
    
    # 주문 문장에서 모든 숫자 및 한글 숫자 정보를 추출하고 변환하는 함수
    def extract_and_convert_numbers(self, order_sentence):
        all_numbers_iter = re.finditer(r"\d+|하나|둘|셋|넷|다섯|여섯|일곱|여덟|아홉|열", order_sentence)
        converted_numbers = [(self.convert_to_number(match), match.span()) for match in all_numbers_iter]
        return converted_numbers 

    def extract_options(self, order_sentence):
        # 정규 표현식을 사용하여 주문 문장에서 정보 추출
        menu_name_match = re.search(self.create_menu_pattern().pattern, order_sentence) # 정규 표현식 문자열 생성
        all_numbers = list(self.extract_and_convert_numbers(order_sentence)) # 모든 iter + 한글 숫자 정보 추출
        temperature_match = re.search(r"차갑게|차가운|아이스|시원한|핫|뜨거운|따뜻하게|따뜻한", order_sentence) # 모든 온도 정보 추출
        size_match = re.search(r"큰|중간|작은|대|중|소|S|M|L", order_sentence) # 사이즈 추출
        shot_word_match = re.search(r"샷", order_sentence) # 샷 span 위치 추출

        """
        샷 추가 ~ 메뉴 수량 언급 ~ 경우의 수
        -1) 아메리카노에 샷 추가 해주세요 (iter X)
        2) 아메리카노 1잔에 샷 추가 해주세요 (iter-수량 1)
        3) 아메리카노에 샷 1개 추가 해주세요 (iter-샷 1)
        -4) 아메리카노 1잔에 샷 1개 추가해주세요 (iter-수량 1, 샷1)
        -5) 아메리카노 1개 주세요 (iter-수량 1)
        -6) 아메리카노 주세요 (iter X)
        7) 아메리카노 하나에 샷 하나 추가해주세요 (=4)
        8) 아메리카노 하나에 샷 추가 해주세요 (iter-수량 1)
        9) 아메리카노에 샷 추가 1잔 주세요 -> 처리 오류 발생... !!!!
        10) 아메리카노에 샷 하나 넣어주세요 -> 샷-추가 사이의 iter 탐지 불가능. (기준으로 하기엔 미달)
        11) 2개 샷 넣은 아메리카노 1개 넣어주세요
        """

        # "샷" 글자와 가장 가까운 숫자 찾기 (re에서 추출된 span 위치 활용)
        if shot_word_match and all_numbers: #2, 3, 4
            shot_index = shot_word_match.start() # span 정보 추출
            closest_list = sorted(all_numbers, key=lambda x: abs(x[1][0] - shot_index)) # 가장 샷 span과 가까운 순으로 정렬

            if len(all_numbers) != 1: # 3, 4
                if shot_index < closest_list[0][1][0]: # 가장 거리가 가까운 숫자가 "샷" 뒤에 숫자가 있는 경우, 샷 추가 수량으로 간주
                    extra_shot_quantity = int(closest_list[0][0]) # 샷과 가까운 iter 찾아 저장
                    closest_list.pop(0)  # 저장했던 샷과 관련된 숫자를 리스트에서 제거 (거리순으로 정렬했기 때문에 맨 앞 요소 삭제)
                else: #가장 거리가 가까운 숫자가 "샷" 앞에 오는 경우 따로 처리 필요. 수가 여러 개 인 경우 (11번 경우)
                    extra_shot_quantity = "TBD" # To Be Determined = 나중에 결정될 필요 있음 (다시 물어보는 변수)
                    
            elif len(all_numbers) == 1: #샷 글자는 있으나, 숫자가 1개인 경우
                if shot_index < closest_list[0][1][0]: # 숫자가 1개 있는 경우, 샷과 가까운 수를 샷 추가 수량으로 간주
                    # "샷" 단어 뒤에 숫자가 있는 경우, 해당 숫자를 샷 추가 수량으로 간주
                    extra_shot_quantity = int(closest_list[0][0])
                    closest_list.pop(0)
                else: #샷 글자 존재. 샷과 가까운 수가 샷 앞에 존재하는 경우, 샷 되묻기 (잔 수로 말한 수 일 수 있음) 9번 경우
                    "TBD"
        elif shot_word_match: # 1
            extra_shot_quantity = 1 # 샷 글자가 존재하나, 숫자가 없는 경우 (ex. 샷 추가)
        else: # 5, 6
            extra_shot_quantity = 0  # 샷 추가가 없는 경우
    
        # 메뉴 수량 찾기 (남은 숫자 중 첫 번째 숫자 사용)
        if extra_shot_quantity == "TBD":
            quantity_match = "TBD"
        else: #이 부분 작동안됨!!!!!!!!!!!
            quantity_match = int(closest_list[0][0]) if closest_list else 1  # 수량 기본값: 1 
    
        # 온도와 사이즈 매칭 정보를 처리
        temperature = (
            "아이스" if temperature_match and re.search(r"차갑게|차가운|아이스|시원한", temperature_match.group(0)) 
            else "핫" if temperature_match and re.search(r"따뜻한|뜨거운|핫", temperature_match.group(0))
            else "TBD"
        )
        # default 값 들어가는 것 처리해야함!!!!!!!!
        size = ("L" if size_match and re.search(r"큰|대|L", size_match.group(0)) else 
                "M" if size_match and re.search(r"중간|중|M", size_match.group(0)) else 
                "S" if size_match and re.search(r"작은|소|S", size_match.group(0)) else "TBD"
               )
        
        # 주문 객체 생성
        order = Order(menu_name=menu_name_match.group(0) if menu_name_match else "TBD",
                      quantity=quantity_match,
                      temperature=temperature,
                      size=size,
                      extra_shot=extra_shot_quantity)

        return order
        
    def add_order(self):
        print(f'<메뉴 입력 모드>\n* 끝내려면 enter 입력\n')
        while True: #주문을 종료하고 싶을 때까지 반복
            #while(1): #주문 완성될 때까지 반복
            order_sentence = input("한 주문씩 입력하세요: ")
            if not order_sentence:
                break
            extracted_order = self.extract_options(order_sentence) # options 추출
            #completed_order = complete_options(extracted_order) # Completed order 생성
        
            self.orders.append(extracted_order)
            print(f"주문이 추가되었습니다: {extracted_order.EO1}, 수량: {extracted_order.EO2}, 온도: {extracted_order.EO3}, 사이즈: {extracted_order.EO4}")
            if extracted_order.DO1 != 0:
                print(f"추가 샷: {extracted_order.DO1}")

        print(f'최종 주문 큐: {self.orders}')

    def try_object(self, obj): # 음료를 찾는 try_object 함수
        self.yes
        for attrib in obj.attrib_list:
            # 이미 yes나 no 리스트에 있는지 확인
            if attrib.attrib in self.yes or attrib.attrib in self.no:
                continue  # 이미 처리된 속성은 건너뜀
            answer = input(f"{attrib.attrib} 괜찮으신가요? (y/n): ").lower()
            if answer == 'n':
                self.no.append(attrib.attrib)
                return False
            elif answer == 'y':
                self.yes.append(attrib.attrib)
        return True

In [9]:
def main():
    system = ExpertSystem()
    while True:
        ch = input("입력, 찾기, 주문, 종료: ").lower()
        if ch == '입력':
            system.enter()
        elif ch == '찾기':
            system.reset_yes_no()
            system.query()
        elif ch == '주문':
            system.add_order()
        elif ch == '종료':
            break
        else:
            print('다시 입력하세요.')

In [10]:
if __name__ == "__main__":
    main()

In [ ]:
#     따뜻한 카라멜 마끼야또 1개 M으로 샷 셋 추가